# Visualization of Burgers Equation Results

This notebook provides visualization tools for analyzing results from the Burgers equation solver.

## Features

- **Solution Evolution**: Time-dependent plots showing wave steepening
- **Shock Wave Analysis**: Multi-panel analysis of discontinuities
- **Gradient Detection**: Identify shock formation through gradient analysis
- **Comparison Tools**: Compare sequential vs parallel results

## Data Sources

Load result files (`.npz` format) from:
- **Sequential solver**: `results/` directory (from `1_sequential_rusanov.ipynb`)
- **Parallel solver**: `plgrid_results/` directory (from PLGrid runs)

## Import Dependencies

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re

# Configure matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Configuration

In [19]:
# Directories
SEQUENTIAL_DIR = Path("results")
PARALLEL_DIR = Path("plgrid_results")
PLOTS_DIR = Path("plots")
PLOTS_DIR.mkdir(exist_ok=True)

print(f"Configuration:")
print(f"  Sequential results: {SEQUENTIAL_DIR}")
print(f"  Parallel results: {PARALLEL_DIR}")
print(f"  Plots directory: {PLOTS_DIR}")

Configuration:
  Sequential results: results
  Parallel results: plgrid_results
  Plots directory: plots


## Discover Available Grid Sizes

Automatically find all grid sizes that have sequential results.

In [20]:
# Find all sequential result files
seq_files = sorted(SEQUENTIAL_DIR.glob("results_sequential_nx*.npz"))

# Extract grid sizes
grid_sizes = []
for f in seq_files:
    match = re.search(r'nx(\d+)', f.name)
    if match:
        grid_sizes.append(int(match.group(1)))

grid_sizes = sorted(grid_sizes)

if len(grid_sizes) == 0:
    print("ERROR: No sequential result files found!")
    print("Please run 1_sequential_rusanov.ipynb first.")
else:
    print(f"\nFound {len(grid_sizes)} grid size(s): {grid_sizes}")
    print(f"Will generate visualizations for each grid size.")


Found 3 grid size(s): [1200, 2400, 4800]
Will generate visualizations for each grid size.


## Load All Sequential Results

In [21]:
# Dictionary to store all results
all_results = {}

for nx in grid_sizes:
    seq_file = SEQUENTIAL_DIR / f"results_sequential_nx{nx}.npz"
    
    if seq_file.exists():
        data = np.load(seq_file)
        all_results[nx] = {
            'x': data['x'],
            'u_final': data['u_final'],
            'snapshots': data['snapshots'],
            'times': data['times']
        }
        
        print(f"Loaded nx={nx:4d}: {len(data['x']):5d} grid points, {len(data['snapshots']):2d} snapshots")
    else:
        print(f"WARNING: File not found: {seq_file}")

print(f"\n[OK] Loaded {len(all_results)} grid size(s)")

Loaded nx=1200:  1200 grid points, 12 snapshots
Loaded nx=2400:  2400 grid points, 12 snapshots
Loaded nx=4800:  4800 grid points, 12 snapshots

[OK] Loaded 3 grid size(s)


## Plot 1: Solution Evolution (All Grid Sizes)

Generate solution evolution plots for each grid size.

In [22]:
print("Generating solution evolution plots...\n")

for nx in sorted(all_results.keys()):
    result = all_results[nx]
    x = result['x']
    snapshots = result['snapshots']
    times = result['times']
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Use colormap for time progression
    colors = plt.cm.viridis(np.linspace(0, 1, len(snapshots)))
    
    for i, (snapshot, t) in enumerate(zip(snapshots, times)):
        ax.plot(x, snapshot, color=colors[i], linewidth=2,
               label=f't = {t:.4f}', alpha=0.7)
    
    ax.set_xlabel('x', fontsize=14)
    ax.set_ylabel('u(x, t)', fontsize=14)
    ax.set_title(f'Burgers Equation Solution Evolution (nx={nx})\n(Rusanov Method)', 
                fontsize=16, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='best', fontsize=10, ncol=2)
    
    plt.tight_layout()
    output_file = PLOTS_DIR / f'solution_evolution_nx{nx}.png'
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    print(f"  Saved: {output_file}")
    plt.close()

print(f"\n[OK] Generated {len(all_results)} solution evolution plots")

Generating solution evolution plots...

  Saved: plots/solution_evolution_nx1200.png
  Saved: plots/solution_evolution_nx2400.png
  Saved: plots/solution_evolution_nx4800.png

[OK] Generated 3 solution evolution plots


## Plot 2: Shock Wave Analysis (All Grid Sizes)

Generate comprehensive 5-panel shock wave analysis for each grid size.

In [23]:
print("Generating shock wave analysis plots...\n")

for nx in sorted(all_results.keys()):
    result = all_results[nx]
    x = result['x']
    snapshots = result['snapshots']
    times = result['times']
    
    n_snapshots = len(snapshots)
    
    # Create figure with multiple subplots
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)
    
    # 1. Solution evolution (top)
    ax1 = fig.add_subplot(gs[0, :])
    colors = plt.cm.viridis(np.linspace(0, 1, n_snapshots))
    for i, (snapshot, t) in enumerate(zip(snapshots, times)):
        ax1.plot(x, snapshot, color=colors[i], linewidth=2, alpha=0.7)
    ax1.set_xlabel('x', fontsize=12)
    ax1.set_ylabel('u(x, t)', fontsize=12)
    ax1.set_title('Solution Evolution', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # 2. Initial vs Final comparison
    ax2 = fig.add_subplot(gs[1, 0])
    ax2.plot(x, snapshots[0], 'b-', linewidth=2, label=f't = {times[0]:.4f}')
    ax2.plot(x, snapshots[-1], 'r-', linewidth=2, label=f't = {times[-1]:.4f}')
    ax2.set_xlabel('x', fontsize=12)
    ax2.set_ylabel('u(x, t)', fontsize=12)
    ax2.set_title('Initial vs Final State', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Gradient analysis for shock detection
    ax3 = fig.add_subplot(gs[1, 1])
    for i in [0, n_snapshots//2, n_snapshots-1]:
        gradient = np.gradient(snapshots[i], x)
        ax3.plot(x, np.abs(gradient), linewidth=2,
                label=f't = {times[i]:.4f}', alpha=0.7)
    ax3.set_xlabel('x', fontsize=12)
    ax3.set_ylabel('|du/dx|', fontsize=12)
    ax3.set_title('Gradient Magnitude (Shock Indicator)', fontsize=14, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.set_yscale('log')
    
    # 4. Heatmap of solution evolution
    ax4 = fig.add_subplot(gs[2, 0])
    T, X = np.meshgrid(times, x)
    contour = ax4.contourf(X, T, snapshots.T, levels=20, cmap='RdYlBu_r')
    plt.colorbar(contour, ax=ax4, label='u(x, t)')
    ax4.set_xlabel('x', fontsize=12)
    ax4.set_ylabel('t', fontsize=12)
    ax4.set_title('Space-Time Evolution (Heatmap)', fontsize=14, fontweight='bold')
    
    # 5. Maximum gradient over time (shock strength evolution)
    ax5 = fig.add_subplot(gs[2, 1])
    max_gradients = []
    for snapshot in snapshots:
        gradient = np.abs(np.gradient(snapshot, x))
        max_gradients.append(np.max(gradient))
    ax5.plot(times, max_gradients, 'ko-', linewidth=2, markersize=6)
    ax5.set_xlabel('Time', fontsize=12)
    ax5.set_ylabel('Max |du/dx|', fontsize=12)
    ax5.set_title('Shock Strength Evolution', fontsize=14, fontweight='bold')
    ax5.grid(True, alpha=0.3)
    
    # Add shock detection annotation
    final_max_gradient = max_gradients[-1]
    if final_max_gradient > 10.0:
        shock_text = "Strong Shock Waves Detected!"
        color = 'red'
    elif final_max_gradient > 2.0:
        shock_text = "Moderate Discontinuities"
        color = 'orange'
    else:
        shock_text = "Smooth Solution"
        color = 'green'
    
    fig.text(0.5, 0.02, shock_text, ha='center', fontsize=14,
            fontweight='bold', color=color)
    
    plt.suptitle(f'Burgers Equation: Shock Wave Analysis (nx={nx})\n(Rusanov Method)',
                fontsize=18, fontweight='bold', y=0.995)
    
    output_file = PLOTS_DIR / f'shock_wave_analysis_nx{nx}.png'
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    print(f"  Saved: {output_file}")
    plt.close()

print(f"\n[OK] Generated {len(all_results)} shock wave analysis plots")

Generating shock wave analysis plots...

  Saved: plots/shock_wave_analysis_nx1200.png
  Saved: plots/shock_wave_analysis_nx2400.png
  Saved: plots/shock_wave_analysis_nx4800.png

[OK] Generated 3 shock wave analysis plots


## Shock Detection Summary (All Grid Sizes)

In [24]:
print("\n" + "="*80)
print("SHOCK WAVE ANALYSIS SUMMARY")
print("="*80)
print(f"{'Grid Size':>12} | {'Max Gradient':>14} | {'Shock Location':>16} | {'Classification':>20}")
print("-"*80)

for nx in sorted(all_results.keys()):
    result = all_results[nx]
    x = result['x']
    u_final = result['u_final']
    
    gradients = np.abs(np.gradient(u_final, x))
    max_gradient = np.max(gradients)
    shock_location = x[np.argmax(gradients)]
    
    if max_gradient > 10.0:
        classification = "Strong shock"
    elif max_gradient > 2.0:
        classification = "Moderate shock"
    else:
        classification = "Smooth"
    
    print(f"{nx:>12d} | {max_gradient:>14.6f} | {shock_location:>16.6f} | {classification:>20}")

print("="*80)


SHOCK WAVE ANALYSIS SUMMARY
   Grid Size |   Max Gradient |   Shock Location |       Classification
--------------------------------------------------------------------------------
        1200 |       0.450057 |         0.750000 |               Smooth
        2400 |       0.451044 |         0.750000 |               Smooth
        4800 |       0.451537 |         0.750000 |               Smooth


## Compare Sequential vs Parallel (All Grid Sizes)

For each grid size, compare sequential vs parallel P=1 results to verify correctness.

## Summary

This notebook automatically generates comprehensive visualizations for **ALL available grid sizes**.

### Generated Plots Per Grid Size:
1. **Solution Evolution** - Wave development over time
2. **Shock Wave Analysis** - Comprehensive 5-panel analysis
3. **Sequential vs Parallel** - Verification comparison (if PLGrid data available)

### Total Files Created:
- `plots/solution_evolution_nx{size}.png`
- `plots/shock_wave_analysis_nx{size}.png`
- `plots/comparison_seq_vs_par_nx{size}.png`

**Next Steps:**
- Run performance analysis with `4_performance_analysis.ipynb`
- Compare scaling behavior across different grid sizes
- Use plots in your report/presentation

In [25]:
print("\nGenerating sequential vs parallel comparison plots...\n")

comparison_results = []

for nx in sorted(all_results.keys()):
    result = all_results[nx]
    x = result['x']
    u_final = result['u_final']
    
    # Try to load parallel result with P=1
    parallel_file = PARALLEL_DIR / f"burgers_nx{nx}_P1.npz"
    
    if parallel_file.exists():
        try:
            par_data = np.load(parallel_file)
            
            x_par = par_data['x']
            u_par = par_data['u_final']
            
            # Interpolate if grids are different
            if len(x) != len(x_par):
                u_par_interp = np.interp(x, x_par, u_par)
            else:
                u_par_interp = u_par
            
            # Create comparison plot
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
            
            # Solution comparison
            ax1.plot(x, u_final, 'b-', linewidth=2, label='Sequential (local)')
            ax1.plot(x_par, u_par, 'r--', linewidth=2, label='Parallel P=1 (PLGrid)', alpha=0.7)
            ax1.set_xlabel('x', fontsize=14)
            ax1.set_ylabel('u(x, t)', fontsize=14)
            ax1.set_title(f'Sequential vs Parallel (P=1) Solution (nx={nx})', 
                         fontsize=16, fontweight='bold')
            ax1.legend(fontsize=12)
            ax1.grid(True, alpha=0.3)
            
            # Error analysis
            error = np.abs(u_final - u_par_interp)
            ax2.plot(x, error, 'g-', linewidth=2)
            ax2.set_xlabel('x', fontsize=14)
            ax2.set_ylabel('|Error|', fontsize=14)
            ax2.set_title('Absolute Error', fontsize=16, fontweight='bold')
            ax2.grid(True, alpha=0.3)
            ax2.set_yscale('log')
            
            # Statistics
            max_error = np.max(error)
            mean_error = np.mean(error)
            fig.text(0.5, 0.02,
                    f'Max Error: {max_error:.2e}  |  Mean Error: {mean_error:.2e}',
                    ha='center', fontsize=12, fontweight='bold')
            
            plt.tight_layout()
            output_file = PLOTS_DIR / f'comparison_seq_vs_par_nx{nx}.png'
            plt.savefig(output_file, dpi=300, bbox_inches='tight')
            print(f"  Saved: {output_file}")
            plt.close()
            
            # Store comparison results
            comparison_results.append({
                'nx': nx,
                'max_error': max_error,
                'mean_error': mean_error
            })
            
        except Exception as e:
            print(f"  WARNING: Error processing nx={nx}: {e}")
    else:
        print(f"  SKIP: Parallel file not found for nx={nx}")

if len(comparison_results) > 0:
    print(f"\n[OK] Generated {len(comparison_results)} comparison plots")
else:
    print(f"\n[INFO] No parallel results available for comparison yet.")
    print(f"       Run PLGrid experiments first to generate comparison plots.")


Generating sequential vs parallel comparison plots...



/var/folders/6_/b9qmtg5j1pxcmnj2dxsbcd240000gn/T/ipykernel_70956/3058188899.py:46: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  ax2.set_yscale('log')


  Saved: plots/comparison_seq_vs_par_nx1200.png
  Saved: plots/comparison_seq_vs_par_nx2400.png
  Saved: plots/comparison_seq_vs_par_nx4800.png

[OK] Generated 3 comparison plots


## Verification Summary (All Grid Sizes)

In [26]:
print("\nGenerating sequential vs parallel comparison plots...\n")

comparison_results = []

for nx in sorted(all_results.keys()):
    result = all_results[nx]
    x = result['x']
    u_final = result['u_final']
    
    # Try to load parallel result with P=1
    parallel_file = PARALLEL_DIR / f"burgers_nx{nx}_P1.npz"
    
    if parallel_file.exists():
        try:
            par_data = np.load(parallel_file)
            
            x_par = par_data['x']
            u_par = par_data['u_final']
            
            # Interpolate if grids are different
            if len(x) != len(x_par):
                u_par_interp = np.interp(x, x_par, u_par)
            else:
                u_par_interp = u_par
            
            # Create comparison plot
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
            
            # Solution comparison
            ax1.plot(x, u_final, 'b-', linewidth=2, label='Sequential (local)')
            ax1.plot(x_par, u_par, 'r--', linewidth=2, label='Parallel P=1 (PLGrid)', alpha=0.7)
            ax1.set_xlabel('x', fontsize=14)
            ax1.set_ylabel('u(x, t)', fontsize=14)
            ax1.set_title(f'Sequential vs Parallel (P=1) Solution (nx={nx})', 
                         fontsize=16, fontweight='bold')
            ax1.legend(fontsize=12)
            ax1.grid(True, alpha=0.3)
            
            # Error analysis
            error = np.abs(u_final - u_par_interp)
            ax2.plot(x, error, 'g-', linewidth=2)
            ax2.set_xlabel('x', fontsize=14)
            ax2.set_ylabel('|Error|', fontsize=14)
            ax2.set_title('Absolute Error', fontsize=16, fontweight='bold')
            ax2.grid(True, alpha=0.3)
            ax2.set_yscale('log')
            
            # Statistics
            max_error = np.max(error)
            mean_error = np.mean(error)
            fig.text(0.5, 0.02,
                    f'Max Error: {max_error:.2e}  |  Mean Error: {mean_error:.2e}',
                    ha='center', fontsize=12, fontweight='bold')
            
            plt.tight_layout()
            output_file = PLOTS_DIR / f'comparison_seq_vs_par_nx{nx}.png'
            plt.savefig(output_file, dpi=300, bbox_inches='tight')
            print(f"  Saved: {output_file}")
            plt.close()
            
            # Store comparison results
            comparison_results.append({
                'nx': nx,
                'max_error': max_error,
                'mean_error': mean_error
            })
            
        except Exception as e:
            print(f"  WARNING: Error processing nx={nx}: {e}")
    else:
        print(f"  SKIP: Parallel file not found for nx={nx}")

if len(comparison_results) > 0:
    print(f"\n[OK] Generated {len(comparison_results)} comparison plots")
else:
    print(f"\n[INFO] No parallel results available for comparison yet.")
    print(f"       Run PLGrid experiments first to generate comparison plots.")


Generating sequential vs parallel comparison plots...



/var/folders/6_/b9qmtg5j1pxcmnj2dxsbcd240000gn/T/ipykernel_70956/3058188899.py:46: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  ax2.set_yscale('log')


  Saved: plots/comparison_seq_vs_par_nx1200.png
  Saved: plots/comparison_seq_vs_par_nx2400.png
  Saved: plots/comparison_seq_vs_par_nx4800.png

[OK] Generated 3 comparison plots
